In [31]:
# conda install -c conda-forge xgboost

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [33]:
xtrain = pd.read_csv("../data/Airbnb_xtrain_Version2.csv",  index_col=0)
ytrain = pd.read_csv("../data/Airbnb_ytrainVersion2.csv", header=None)
xtrain.head()

,tfa_year,tfa_month,tfa_day,tfa_wd_1,tfa_wd_2,tfa_wd_3,tfa_wd_4,tfa_wd_5,tfa_wd_6,tfa_wd_7,...,c_447,c_448,c_449,c_450,c_451,c_452,c_453,c_454,c_455,c_456
language,,,,,,,,,,,,,,,,,,,,,
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,11.0,27.0,15.0,4.0,9.0,10.0,14.0,5.0,1.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,2.0,2.0,1.0,1.0,0.0,0.0,0.0
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
en,2014.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,16.0,22.0,28.0,29.0,7.0,6.0,7.0,7.0,1.0,0.0


In [34]:
ytrain

,0
0,country_destination
1,other
2,NDF
3,NDF
4,NDF
...,...
73808,US
73809,NDF
73810,US
73811,NDF


In [35]:
ytrain_le = LabelEncoder().fit_transform(ytrain.values.ravel())
ytrain_new = ytrain_le       # Encode label

X_scaler = StandardScaler()
xtrain_new = X_scaler.fit_transform(xtrain)  # Normalization

In [ ]:
pd.DataFrame(xtrain_new).to_csv("../data/Airbnb_xtrain_v3.csv")
pd.DataFrame(ytrain_new).to_csv("../data/Airbnb_ytrain_v3.csv")

In [ ]:
NUM_XGB = 10

params = {'num_class': 12,
          'eta':0.1,
          'objective':'multi:softprob'}
train_xgb = xgb.DMatrix(xtrain_new, label= ytrain_new)
bst = xgb.train(params, train_xgb, NUM_XGB)

In [ ]:
fig,ax = plt.subplots(figsize=(10,15))
xgb.plot_importance(bst,height=0.3,ax=ax,max_num_features=50)
plt.show()

In [ ]:
fig.savefig('importance', dpi = 100)

In [ ]:
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0} \t {1} \t q \n'.format(i, feat))
        i = i + 1
    outfile.close()


In [ ]:
import operator
features = pd.DataFrame(xtrain_new).columns
pd.DataFrame(xtrain_new).head()


In [ ]:
ceate_feature_map(features)
 
importance = bst.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
 
df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()
df.to_csv("feat_importance.csv", index=False)

In [ ]:
# select the most important n features
df_selected=df.head(300)
data_index = df_selected['feature'].tolist()
feature_index = []
for index in data_index:
    int_index = int(index)
    feature_index.append(int_index)
print(feature_index)

In [ ]:
xtest = pd.read_csv("Airbnb_xtest_Version2.csv", index_col=0)
xtest.head()

In [ ]:
xtrain_v4 = xtrain.iloc[:, feature_index]
xtest_v4 = xtest.iloc[:, feature_index]
xtrain_v4.to_csv("../data/Airbnb_xtrain_v4.csv")
xtest_v4.to_csv("../data/Airbnb_xtest_v4.csv")

In [ ]:
xtrain_v4.head()